In [2]:
api_key = "API" #<--- Chave API do Google

In [3]:
from apiclient.discovery import build
import pandas as pd

In [4]:
youtube = build('youtube', 'v3', developerKey=api_key)
ID = "ID" #<----ID do video para capturar os comentários

box = [['Name', 'Comment', 'Time', 'Likes', 'Reply Count']]

In [5]:
def scrape_comments_with_replies():
    
    data = youtube.commentThreads().list(part='snippet', videoId=ID, maxResults='100', textFormat="plainText").execute()

    for i in data["items"]:

        name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        replies = i["snippet"]['totalReplyCount']

        box.append([name, comment, published_at, likes, replies])

        totalReplyCount = i["snippet"]['totalReplyCount']

        if totalReplyCount > 0:

            parent = i["snippet"]['topLevelComment']["id"]

            data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                            textFormat="plainText").execute()

            for i in data2["items"]:
                name = i["snippet"]["authorDisplayName"]
                comment = i["snippet"]["textDisplay"]
                published_at = i["snippet"]['publishedAt']
                likes = i["snippet"]['likeCount']
                replies = ""

                box.append([name, comment, published_at, likes, replies])

    while ("nextPageToken" in data):

        data = youtube.commentThreads().list(part='snippet', videoId=ID, pageToken=data["nextPageToken"],
                                             maxResults='100', textFormat="plainText").execute()

        for i in data["items"]:
            name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
            comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
            published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
            likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
            replies = i["snippet"]['totalReplyCount']

            box.append([name, comment, published_at, likes, replies])

            totalReplyCount = i["snippet"]['totalReplyCount']

            if totalReplyCount > 0:

                parent = i["snippet"]['topLevelComment']["id"]

                data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                                textFormat="plainText").execute()

                for i in data2["items"]:
                    name = i["snippet"]["authorDisplayName"]
                    comment = i["snippet"]["textDisplay"]
                    published_at = i["snippet"]['publishedAt']
                    likes = i["snippet"]['likeCount']
                    replies = ''

                    box.append([name, comment, published_at, likes, replies])

    df = pd.DataFrame({'Name': [i[0] for i in box], 'Comment': [i[1] for i in box], 'Time': [i[2] for i in box],
                       'Likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box]})

    df.to_csv('youtube-comments.csv', encoding="utf-8", mode='a' ,index=False, header=False) #<--- Gera o CSV dos comentários

    return df


scrape_comments_with_replies()

,Name,Comment,Time,Likes,Reply Count
0,Name,Comment,Time,Likes,Reply Count
1,Marcela Cunha,Ele terminou o NOIVADO comigo porque nossa rel...,2022-07-14T04:26:36Z,0,0
2,wagnÓSTICO.,"Nao tem botão, mas tem remédio fisico, descobr...",2022-07-09T17:04:08Z,0,0
3,Eliza Jpes,"E quando a gente amou por muitos, muitos anos?...",2022-07-09T14:55:34Z,0,1
4,lipe guilherme,Manda o zap q te tigo por la kkk,2022-07-13T23:34:01Z,0,
...,...,...,...,...,...
636,Belinhagamer999,@O 4° SOBREVIVENTE HUNK EQUIPE ALFA exato as v...,2022-04-23T15:51:44Z,3,
637,Belinhagamer999,@Evelyn santos fazem isso pra chamar atenção,2022-04-23T15:50:16Z,0,
638,O 4° SOBREVIVENTE HUNK EQUIPE ALFA,Sim belinhagamer: Exatamente as pessoas de ho...,2022-04-23T15:09:38Z,8,
639,Evelyn santos,"Exatamente, escolhem uma coisa e colocam isso ...",2022-04-22T23:11:32Z,8,
